In [1]:
import sys
!{sys.executable} -m pip install xarray zarr itk>=5.1 dask[array] toolz itkwidgets

You are using pip version 9.0.1, however version 20.2.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import itk
import xarray as xr
import numpy as np
from numcodecs import Blosc, blosc
import zarr
from urllib.request import urlretrieve
import os

from itkwidgets import view, compare

In [3]:
ccf_filename = '1.nii.gz'
dataset_name = 'miqaSample1'
store_name = 'miqaSample1.zarr'
units = 'mm'

In [4]:
image = itk.imread(ccf_filename)

In [5]:
view(image, vmax=300, gradient_opacity=0.1, units='mm')

Viewer(geometries=[], gradient_opacity=0.1, point_sets=[], rendered_image=<itk.itkImagePython.itkImageSS3; pro…

In [6]:
print(itk.size(image))
print(itk.spacing(image))

itkSize3 ([160, 192, 192])
itkVectorD3 ([1, 1.33333, 1.33333])


In [7]:
# Available in ITK 5.1 RC 2 and later
image_da = itk.xarray_from_image(image)
image_da

<xarray.DataArray (z: 192, y: 192, x: 160)>
array([[[3, 0, 2, ..., 1, 1, 2],
        [0, 0, 1, ..., 1, 2, 1],
        [3, 2, 0, ..., 1, 1, 1],
        ...,
        [2, 1, 2, ..., 2, 2, 1],
        [2, 1, 2, ..., 0, 1, 3],
        [0, 0, 0, ..., 0, 0, 0]],

       [[2, 2, 2, ..., 2, 2, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [0, 0, 2, ..., 0, 1, 1],
        ...,
        [0, 0, 1, ..., 2, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [0, 0, 0, ..., 0, 0, 0]],

       [[3, 2, 2, ..., 1, 1, 2],
        [0, 1, 0, ..., 1, 3, 0],
        [0, 0, 1, ..., 0, 2, 3],
        ...,
        [2, 2, 3, ..., 3, 0, 2],
        [1, 2, 2, ..., 0, 0, 2],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[3, 1, 0, ..., 2, 2, 2],
        [1, 3, 3, ..., 0, 2, 0],
        [1, 1, 1, ..., 4, 3, 0],
        ...,
        [1, 2, 2, ..., 2, 2, 2],
        [1, 1, 1, ..., 2, 2, 1],
        [0, 0, 0, ..., 0, 0, 0]],

       [[1, 3, 1, ..., 4, 0, 1],
        [1, 0, 0, ..., 2, 2, 3],
        [2, 0, 1, ..., 4,

In [8]:
image_da.attrs['units'] = units
image_da.attrs

OrderedDict([('direction', array([[ 1.,  0.,  0.],
                     [ 0., -1.,  0.],
                     [ 0.,  0.,  1.]])), ('units', 'mm')])

In [9]:
# multi-resolution pyramid
pyramid = [image_da]
reduced = image
while not np.all(np.array(itk.size(reduced)) < 32):
    level = len(pyramid)
    shrink_factors = [2**level]*3
    reduced = itk.bin_shrink_image_filter(image, shrink_factors=shrink_factors)
    reduced_da = itk.xarray_from_image(reduced)
    reduced_da.attrs['units'] = units
    print('level', level)
    print('origin', itk.origin(reduced))
    print('spacing', itk.spacing(reduced))
    print('size', itk.size(reduced))
    pyramid.append(reduced_da)

level 1
origin itkPointD3 ([-79.5, 127.333, -127.333])
spacing itkVectorD3 ([2, 2.66667, 2.66667])
size itkSize3 ([80, 96, 96])
level 2
origin itkPointD3 ([-78.5, 126, -126])
spacing itkVectorD3 ([4, 5.33333, 5.33333])
size itkSize3 ([40, 48, 48])
level 3
origin itkPointD3 ([-76.5, 123.333, -123.333])
spacing itkVectorD3 ([8, 10.6667, 10.6667])
size itkSize3 ([20, 24, 24])


In [10]:
compare(image, itk.image_from_xarray(pyramid[-1]), mode='y', vmax=300)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…

In [11]:
image_ds = image_da.to_dataset(name=dataset_name)
image_ds

<xarray.Dataset>
Dimensions:      (x: 160, y: 192, z: 192)
Coordinates:
  * y            (y) float64 128.0 129.3 130.7 132.0 ... 378.7 380.0 381.3 382.7
  * z            (z) float64 -128.0 -126.7 -125.3 -124.0 ... 124.0 125.3 126.7
  * x            (x) float64 -80.0 -79.0 -78.0 -77.0 ... 76.0 77.0 78.0 79.0
Data variables:
    miqaSample1  (z, y, x) int16 3 0 2 1 2 1 3 1 1 1 0 ... 0 0 0 0 0 0 0 0 0 0 0

In [12]:
pyramid_group_paths = [""]
for level in range(1, len(pyramid)):
#for level in range(1, 6):
    pyramid_group_paths.append('level_{0}'.format(level))
pyramid_group_paths

['', 'level_1', 'level_2', 'level_3']

In [13]:
store = zarr.DirectoryStore(store_name)

# NOSHUFFLE since we will be visualizing with WebAssembly, which does not currently have support for intrinsics
blosc.use_threads = False
# Crashing (?)
# compressor = Blosc(cname='zstd', clevel=5, shuffle=Blosc.NOSHUFFLE)
compressor = Blosc(clevel=5, shuffle=Blosc.NOSHUFFLE)
chunk_size = 64

In [14]:
image_ds.to_zarr(store,
                 mode='w',
                 compute=True,
                 encoding={dataset_name: {'chunks': [chunk_size]*3, 'compressor': compressor}})

In [15]:
# for level in range(len(pyramid)-1, 0, -1):
for level in range(len(pyramid)-1, 1, -1): # level 1 crashes
# for level in range(1, len(pyramid)):
    print('level', level)
    ds = pyramid[level].to_dataset(name=dataset_name)
    print(ds)
    compressor = Blosc(clevel=5, shuffle=Blosc.NOSHUFFLE)
    ds.to_zarr(store,
               mode='w',
               group=pyramid_group_paths[level],
               compute=True,
               encoding={dataset_name: {'chunks': [chunk_size]*3, 'compressor': compressor}})

level 3
<xarray.Dataset>
Dimensions:      (x: 20, y: 24, z: 24)
Coordinates:
  * y            (y) float64 123.3 134.0 144.7 155.3 ... 336.7 347.3 358.0 368.7
  * z            (z) float64 -123.3 -112.7 -102.0 -91.33 ... 100.7 111.3 122.0
  * x            (x) float64 -76.5 -68.5 -60.5 -52.5 ... 51.5 59.5 67.5 75.5
Data variables:
    miqaSample1  (z, y, x) int16 1 1 1 1 1 1 2 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
level 2
<xarray.Dataset>
Dimensions:      (x: 40, y: 48, z: 48)
Coordinates:
  * y            (y) float64 126.0 131.3 136.7 142.0 ... 360.7 366.0 371.3 376.7
  * z            (z) float64 -126.0 -120.7 -115.3 -110.0 ... 114.0 119.3 124.7
  * x            (x) float64 -78.5 -74.5 -70.5 -66.5 ... 65.5 69.5 73.5 77.5
Data variables:
    miqaSample1  (z, y, x) int16 336 15248 350 0 28688 20661 350 ... 0 0 0 0 0 0


In [16]:
# After all modifications to the store are complete, consolidate the metadata so it is 'cloud-ready'.
zarr.consolidate_metadata(store)

<zarr.hierarchy.Group '/'>

In [17]:
ds = xr.open_zarr(store_name, group='level_3', consolidated=True)

In [18]:
da = ds[dataset_name]
da

<xarray.DataArray 'miqaSample1' (z: 24, y: 24, x: 20)>
dask.array<zarr, shape=(24, 24, 20), dtype=int16, chunksize=(24, 24, 20), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) float64 -76.5 -68.5 -60.5 -52.5 -44.5 ... 51.5 59.5 67.5 75.5
  * y        (y) float64 123.3 134.0 144.7 155.3 ... 336.7 347.3 358.0 368.7
  * z        (z) float64 -123.3 -112.7 -102.0 -91.33 ... 90.0 100.7 111.3 122.0
Attributes:
    units:      mm
    direction:  [[1.0, 0.0, 0.0], [0.0, -1.0, 0.0], [0.0, 0.0, 1.0]]

In [19]:
image_level_3 = itk.image_from_xarray(da)
view(image_level_3)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageSS3; pr…

In [20]:
for level in range(len(pyramid)-1, 1, -1): # level 1 crashes
# for level in range(1, len(pyramid)):
    print('level', level)
    store = zarr.DirectoryStore(store_name + '/' + pyramid_group_paths[level])
    # Also consolidate the metadata on the pyramid levels so they can be used independently
    zarr.consolidate_metadata(store)

level 3
level 2
